## Propósito do projeto

##### Nosso propósito é ajudar você a treinar e colocar em prática seus estudos de inglês. Eu forneço frases e histórias com o objetivo de você tentar traduzi-las. Depois, você pode verificar se acertou solicitando a tradução da frase ou história. Além disso, recentemente adicionamos uma seção para aprender alguns conteúdos relativos ao inglês, constantemente o sistema será atualizado com novos conteúdos. Nós surgimos da necessidade de um lugar onde pudéssemos treinar nossos aprendizados de forma prática, traduzindo pequenos textos ou frases, mas que estes estivessem no nosso nível de aprendizado. Muitas das vezes, outros lugares que tinham essas frases e histórias, não possuíam um nível equivalente ao nosso aprendizado. Dai eu surgi, com o objetivo de te ajudar a aprender cada vez mais.

## Importando Bibliotecas

Nesse projeto utilizaremos 7 bibliotecas.

- *Telebot* - responsável por fazer a comunicação com o Telegram.

- *Telebot types* - responsável pela interface de botões no telegram.

- *Openpyxl* - responsável por fazer a comunicação com a base de dados, no caso, uma planilha do excel.

- *MatériasFunctions.Numbers.Numbers* - Funções responsáveis pela parte de aprender números em inglês, importada de outro arquivo dentro do próprio código.

- *FrasesHistorias.Frases* - Funções responsáveis pela parte de exibir frases e histórias, importada de outro arquivo dentro do próprio código.

- *MatériasFunctions.AlphabetPronunciation.AlphabetPronunciation* - Funções responsáveis pela parte de exibir a pronúncia do alfabeto em inglês, importada de outro arquivo dentro do próprio código.

- *OS* - Função responsável por verificar o caminho correto para os arquivos dentro do código.

Antes de importar, devemos fazer o download das bibliotecas!

Versões disponíveis no arquivo requeriments.

In [ ]:
import telebot
from telebot import types
from openpyxl import load_workbook
from MatériasFunctions.Numbers.Numbers import Numbers, VerificarNumberExtenso
from FrasesHistorias.Frases import exibirFrase, exibirHistoria, exibirTraducao, AlterarNivel
from MatériasFunctions.AlphabetPronunciation.AlphabetPronunciation import alphabet, verificar_pronuncia
import os

## Definindo sua chave API

### Criando o chatbot

Para conseguir realizar a comunicação com o Telegram, primeiro devemos criar um chat no telegram, non qual utilizaremos como chatbot

Para isto devemos seguir alguns passos simples:

- Passo 1: Ir até a pesquisa do app Telegram e procurar por BotFather, ele terá o simbolo de verificado

- Passo 2: Apertar o botão Start, para começar uma nova conversa

- Passo 3: Ele vai te retornar uma mensagem com vários comandos, para criar um novo bot, basta selecionar a opção /newbot

- Passo 4: Ele pedirá para que você defina um nome para o bot, no nosso caso, se chama English Bot

- Passo 5: Agora ele pedirá para que você defina um username para o bot. Atençao! o username deve ser terminado com a palavra 'bot'. Exemplo: EnglishBot, English_bot e deve estar disponível.

- Passo 6: Agora ele te retornará uma mensagem com os dados do seu bot, o primeiro link, 't.me/nomedobot' será o link para a conversa com seu chatbot. Nesta mesma mensagem, também haverá a sua chave Api, responsável por permitir a conexão do seu script com o chatbot.

Não compartilhe esta chave API, ela da acesso ao controle do seu chatbot.

### Definindo a chave no script

In [1]:
# Com a chave API em mãos, basta definir a nossa variável para ela

CHAVE_API = "Sua Chave api aqui"

### Iniciando o bot

In [ ]:
# Para iniciar o bot e permitir a comunicação com o Telegram, basta executar o comando abaixo

bot = telebot.TeleBot(CHAVE_API)

# Agora, todas as operações envolvendo o bot devem ser realizadas através da variável 'bot'

### Classe User

Devido a extensão do código ser muito grande, torna-se necessário a implantação de uma classe com as informações do usuário. Isto ocorre, pois sem utilizar classes, teríamos que transitar com várias variáveis por todo o código, de função para função, o que poderia acarretar a alteração de uma variável de forma indevida durante a execução, além de facilitar a compreenssão do que cada variável representa.

A classe User possui todas a informações do usuário (id, frase, traducao, nivel, number, ultimoComando, fraseOrAprender) de forma privada, podendo ser acessada ou alterada através de uma função específica dentro da classe.

In [ ]:
class User(): 
    def __init__(self, id, frase, traducao, nivel, number, ultimoComando, fraseOrAprender): # Define todas as variáveis da classe
        self.__id = id
        self.__frase = frase
        self.__traducao = traducao
        self.__nivel = nivel
        self.__number = number
        self.__ultimoComando = ultimoComando
        self.__fraseOrAprender = fraseOrAprender


    # Funções para retonar os valores de cada variável
    def get_id(self):
        return self.__id
    
    def get_frase(self):
        return self.__frase
    
    def get_traducao(self):
        return self.__traducao
    
    def get_nivel(self):
        return self.__nivel
    
    def get_number(self):
        return self.__number
    
    def get_ultimoComando(self):
        return self.__ultimoComando
    
    def get_fraseOrAprender(self):
        return self.__fraseOrAprender
    
    # Funções para alterar os valores de cada variável
    
    def set_id(self, id):
        self.__id = id

    def set_frase(self, frase):
        self.__frase = frase

    def set_traducao(self, traducao):
        self.__traducao = traducao

    def set_nivel(self, nivel):
        self.__nivel = nivel

    def set_number(self, number):
        self.__number = number

    def set_ultimoComando(self, ultimoComando):
        self.__ultimoComando = ultimoComando

    def set_fraseOrAprender(self, fraseOrAprender):
        self.__fraseOrAprender = fraseOrAprender

### Criando as funções

#### Carregar Planilha

Essa função é responsável por verificar em qual pasta o codigo que esta sendo executado se encontra e define qual o caminho do arquivo procurado. 

Ela recebe como parâmetro o nome do arquivo procurado e retorna o caminho deste arquivo.

Por exemplo:

Este código app.py está na pasta raiz, caso queira acessar o arquivo dos usuário que está na pasta DataBase, essa função irá retornar o caminho absoluto deste arquivo: *'Caminho da pasta raiz\EnglishBot\DataBase\Usuarios.xlsx'*

sem esta função, o código poderia gerar um erro ao tentar acessar um arquivo em local indevido.

In [ ]:
def carregar_planilha(pasta):
    diretorio_atual = os.getcwd()
    pasta_database = os.path.join(diretorio_atual, 'DataBase')
    caminho_arquivo = os.path.join(pasta_database, pasta)
    return caminho_arquivo

#### Alterar Usuário

Função responsável por altualizar a base de dados após todas as alterações feitas no perfil do usuário durante a execução do programa, será sempre a última a ser executada após cada mensagem do usuário.

Essa função recebe como parâmetros o objeto do usuário criado e a linha em que as informações desse determinado usuário estão na base de dados.

Carrega a planilha, altera os dados necessários e ao final, salva a planilha com as alterações.

In [ ]:
def AlterarUsuario(user, linha):
    caminho = carregar_planilha('Usuarios.xlsx')
    planilha = load_workbook(caminho)
    aba_ativa = planilha.active
    aba_ativa[f'A{linha}'] = user.get_id()
    aba_ativa[f'B{linha}'] = user.get_frase()
    aba_ativa[f'C{linha}'] = user.get_traducao()
    aba_ativa[f'D{linha}'] = user.get_nivel()
    aba_ativa[f'E{linha}'] = user.get_number()
    aba_ativa[f'F{linha}'] = user.get_ultimoComando()
    aba_ativa[f'G{linha}'] = user.get_fraseOrAprender()
    planilha.save(caminho)

#### Responder

Esta função é responsável por fazer a comunicação com o telegram enviando as mensagens necessárias.

Ela recebe como parâmetro 4 componentes:

- *Id do usuário* - Para identificar o chat para qual a mensagem deve ser enviada.

- *Resposta* - Mensagem que deve ser enviada ao usuário.

- *buttons* - Uma lista com todos os botões que devem ser exibidos junto com a mensagem, cada botão contém uma lista comk o nome que será exibido e o callback(mensagem ou comando retornado ao apertar o botão).

- *qtd* - Variável contendo a quantidade de botões que serão exibidos em cada linha.

Nessa função, é criado uma variável markup. Esta variável se refere ao grupo de botões, que serão adicionados posteriormente.

Após isso, os botões são criados e adicionados ao conjunto(markup) e envidados para o usuário.


In [ ]:
def responder(id, resposta, buttons, qtd):
    btn = []
    markup = types.InlineKeyboardMarkup(row_width=qtd)
    for bt in buttons:
        button = types.InlineKeyboardButton(bt[0], callback_data=bt[1])
        btn.append(button)

    markup.add(*btn)

    bot.send_message(id, resposta, reply_markup=markup)   

#### Responder sem Botão

Essa função também é responsável por fazer a comunicação com o telegram enviando mensagens, porém diferente4 da anterior, ela envia somente a mensagem, sem botões. É usada em alguns casos específicos onde é necessário enviar somente uma mensagem ou socilitar algo escrito do usuário.

Recebe como parâmetros apenas 2 elementos:

- *Id do usuário* - Para identificar o chat para qual a mensagem deve ser enviada.

- *Resposta* - Mensagem que deve ser enviada ao usuário.

In [ ]:
def responder_sem_button(id, resposta):
    bot.send_message(id, resposta)   

### Como funciona o código?

No telegram existem mensagens específicas que se comportam de maneiras diferentes das demais mensagens, os comandos. Sempre que adicionados '/' antes de qualquer mensagem, ela se torna um comando no telegram, se comportando semelhand=te a um link clicável. Para diferenciar as mensagens padrão das mensagens que farão alguma ação, todo o código é baseado em comandos, por exemplo.

Ao apertar o botão Aprender inglês, ele retornará o callback(mensagem) como '/Aprender', desta forma, sempre que receber uma mensagem como essa, ele acionará a seção de aprender. 

Todo o código é baseado em comando para evitar erros acidentais, ao pressionar qualquer botão, o telegram retorna essa informação como uma mensagem normal definida no callback do botão e não um comando em si. Por isso, se caso fosse utilizado somente 'Aprender' para acionar uma função, em uma frase aleatória em que o usuário digitasse isso, poderia acionar a função erroneamente, para isso são utilizados os comandos, pois não é comum o usuário digitar '/' antes das palavras.

#### Exibir menu

A função de exibir o menu é chamada toda vez que uma mensagem é recebida e não é nenhum dos comandos do código, ela é chamada como "exceção", ou seja, ao receber uma mensagem, o código verificar se é um comando, caso não seja, exibe o menu.

Essa função recebe como parâmetros o objeto do usuário criado.

Ela exibe uma mensagem de boas vindas e chama a função de responder, passando o id, contido no objeto do usuário, a mensagem, todos os botões que devem ser exibidos e a quantidade em cada linha.

In [ ]:
def exibirMenu(user):
    id = user.get_id()
    resposta = f'Olá, seja muito bem vindo! 👋 \n\nEste é o nosso Menu 🏠'
    responder(id, resposta, [['Frases ou Histórias', '/FraseHistoria'], ['Aprender Inglês', '/Aprender'], ['Nosso Propósito', '/Proposito']], 1)

#### Verificar comando aprender

Essa função é responsável por verificar se os comandos recebidos são referentes a aba de apreder ou a aba de frases e histórias, para chamar as funções específicas de cada caso.

Ela recebe o objeto do usuário e a mensagem para verificação

In [ ]:
def verificarComandoAprender(mensagem,user):
    id = user.get_id()
    ultimoComando = user.get_ultimoComando()

    if mensagem == '/FraseHistoria':
        user.set_fraseOrAprender('Frase')
        verificarComandoFrases(mensagem, user)

    elif mensagem == '/Frase' or mensagem == '/Historia' or mensagem == '/Traducao' or mensagem == '/Nivel':
        user.set_fraseOrAprender('Frase')
        Frases(mensagem, user)
    
    elif mensagem == '/Aprender':
        user.set_ultimoComando('/Aprender')
        resposta =  f'Que bom que você queira aprender! 👋 \n\nSelecione a materia que deseja aprender!'
        responder(id, resposta, [['Alphabet', '/Alfabeto'], ['Numbers', '/Numbers'], ['Menu', '/OK']], 1)

    elif ultimoComando == '/Aprender':
        alfabeto = ['/A', '/B', '/C', '/D', '/E', '/F', '/G', '/H', '/I', '/J', '/K', '/L', '/M', '/N', '/O', '/P', '/Q', '/R', '/S', '/T', '/U', '/V', '/W', '/X', '/Y', '/Z']

        if mensagem == '/Alfabeto' or mensagem in alfabeto or mensagem == '/TreinarAlfabeto' or mensagem == '/Alfabet':
            alphabet(user, mensagem)

        elif mensagem == '/Numbers' or mensagem == '/ConteudoNumbers' or mensagem == '/ExibirNumber' or mensagem == '/ExtensoNumbers' or mensagem == '/ExibirNumberExtenso':
            Numbers(mensagem, user)
        else:  
            user.set_ultimoComando('/OK') 
            exibirMenu(user)
        
    elif mensagem == '/Proposito':
        resposta = f'Olá, que bom que queira saber mais de nós 😊 \n\nNosso propósito é ajudar você a treinar e colocar em prática seus estudos de inglês. Eu forneço frases e histórias com o objetivo de você tentar traduzi-las. Depois, você pode verificar se acertou solicitando a tradução da frase ou história. \n\nNós surgimos da necessidade de um lugar onde pudéssemos treinar nossos aprendizados de forma prática, traduzindo pequenos textos ou frases, mas que estes estivessem no nosso nível de aprendizado. Muitas das vezes, outros lugares que tinham essas frases e histórias, não possuíam um nível equivalente ao nosso aprendizado. Dai eu surgi, com o objetivo de te ajudar a aprender cada vez mais. \n\nFico muito feliz de tê-lo por aqui! 😊😊'
        responder(id, resposta, [['Continuar','/OK']], 1)

    else:
        if ultimoComando == '/ExtensoNumbers':
            VerificarNumberExtenso(mensagem, user)
        
        elif ultimoComando == '/VerificarPronuncia':
            verificar_pronuncia(user, mensagem)	

        else:
            exibirMenu(user)


#### Verificar comando frase

Essa função é responsável por exibir o menu de ações referente a frases os histórias, onde terão todos os comandos relacionados ao assunto.

Ela recebe o objeto do usuário e a mensagem, pois deve verificar se a aba selecionada foi realmente a de frases ou histórias.

In [ ]:
def verificarComandoFrases(mensagem,user):
    id = user.get_id()
    if mensagem == '/FraseHistoria' or mensagem == '/Menu':
        resposta = f'Este é o nosso Menu de Frases e Histórias 🏠'
        responder(id, resposta, [['Exibir Frase', '/Frase'], ['Exibir História', '/Historia'], ['Traduzir Frase/História', '/Traducao'], ['Alterar Nível', '/Nivel'], ['Nosso Propósito', '/Proposito'], ['Menu', '/OK']], 1)

    else:
        Frases(mensagem, user)
 

#### Frases

Essa  função é responsável por verificar todos os comandos referentes a seção de fases e histórias e chamar a função específica para cada ação, seja exibir uma frase, uma história, a tradução etc

Ela recebe como parâmetros a mensagem(comando) recebida e o objeto do usuário

In [ ]:
def Frases(mensagem, user):
    id = user.get_id()
    if mensagem == '/Frase':
        exibirFrase(user)
    
    elif mensagem == '/Historia':
        exibirHistoria(user)
    
    elif mensagem == '/Traducao':
        exibirTraducao(user)
    
    elif mensagem == '/Nivel':
        nivel = 'Nivel'
        AlterarNivel(user, nivel)

    elif mensagem == '/Basico':
        nivel = 'Básico'
        AlterarNivel(user, nivel)
    
    elif mensagem == '/BasicoAvancado':
        nivel = 'Básico Avançado'
        AlterarNivel(user, nivel)
    
    elif mensagem == '/Intermediario':
        nivel = 'Intermediário'
        AlterarNivel(user, nivel)
    
    elif mensagem == '/IntermediarioAvancado':
        nivel = 'Intermediário Avançado'
        AlterarNivel(user, nivel)
    
    elif mensagem == '/Fluente':
        nivel = 'Fluente'
        AlterarNivel(user, nivel)

    elif mensagem == '/Aprender':
        user.set_fraseOrAprender('Aprender')
        verificarComandoAprender(mensagem, user)
        
    elif mensagem == '/Proposito':
        resposta = f'Olá, que bom que queira saber mais de nós 😊 \n\nNosso propósito é ajudar você a treinar e colocar em prática seus estudos de inglês. Eu forneço frases e histórias com o objetivo de você tentar traduzi-las. Depois, você pode verificar se acertou solicitando a tradução da frase ou história. \n\nNós surgimos da necessidade de um lugar onde pudéssemos treinar nossos aprendizados de forma prática, traduzindo pequenos textos ou frases, mas que estes estivessem no nosso nível de aprendizado. Muitas das vezes, outros lugares que tinham essas frases e histórias, não possuíam um nível equivalente ao nosso aprendizado. Dai eu surgi, com o objetivo de te ajudar a aprender cada vez mais. \n\nFico muito feliz de tê-lo por aqui! 😊😊'
        responder(id, resposta, [['Continuar','/OK']], 1)

    else:
        exibirMenu(user)


#### Section

Essa função é responsável por identificar se o usuário escolheu a opção de aprender ou de exibir frases e histórias especificamente. Difirente das funções anteriores, essa não verifica qual o comando recebido para chamar uma determinada função, ela apenas 'Divide o código em duas partes', verificando se o usuário escolheu uma ou outra.

Recebe como parâmetros o objeto, de onde será tirada a decisão do usuário e a mensagem, apenas para enviar a outras funções que utilizam.

In [ ]:
def Section(mensagem, user):
    usuario = user.get_fraseOrAprender()
    if usuario == 'Frase':
        verificarComandoFrases(mensagem, user)
    elif usuario == 'Aprender':
        verificarComandoAprender(mensagem, user)


#### Registrar usuário

Essa função é especificamente para registrar novos usuários na base de dados, ela carrega as tabelas e adiciona uma nova linha com informações temporárias do usuário, que serão modificadas durante a execuçào do programa.

Recebe como parâmetros o objeto do usuário com as informações temporárias.

In [ ]:
def registrarUsuario(user):
    caminho = carregar_planilha('Usuarios.xlsx')
    planilha = load_workbook(caminho)
    aba_ativa = planilha.active
    usuario = [user.get_id(), user.get_frase(), user.get_traducao(), user.get_nivel(), user.get_number(), user.get_ultimoComando(), user.get_fraseOrAprender(), user.get_letra()]
    aba_ativa.append(usuario)
    planilha.save(caminho)

#### User Info

Essa funçào é responsável por acessar as tabelas e retornar todas as informações do usuário, para atualizar o objeto e poder realizar as funções do código com precisão.

Recebe como parâmetros apenas o id, para identificar o usuário na base de dados.

In [ ]:
def UserInfo(id):
    caminho = carregar_planilha('Usuarios.xlsx')
    planilha = load_workbook(caminho)
    aba_ativa = planilha.active
    for celula in aba_ativa['A']:
        if type(celula.value) == int :
            if celula.value == id:
                linha = celula.row
                frase = aba_ativa[f'B{linha}'].value
                traducao = aba_ativa[f'C{linha}'].value
                nivel = aba_ativa[f'D{linha}'].value
                number = aba_ativa[f'E{linha}'].value
                UltimaComando = aba_ativa[f'F{linha}'].value
                comando = aba_ativa[f'G{linha}'].value
                letra = aba_ativa[f'H{linha}'].value
                return [id, frase, traducao, nivel, number, UltimaComando, comando, letra], linha
   

#### Receber Lista de Users

Essa função é responsável por retornar uma lista com todos os ids dos usuários presentes na base de dados.
Essa lista será usada para definir se o usuário já existe na base de dados ou é um usuário novo.

In [ ]:
def receberListaUser():
    lista_id = []
    caminho = carregar_planilha('Usuarios.xlsx')
    planilha = load_workbook(caminho)
    aba_ativa = planilha.active
    for celula in aba_ativa['A']:
      if type(celula.value) == int :
        lista_id.append(celula.value)
    return lista_id

#### Verificar usuário

Esta função é complementar à anterior, é ela quem recebe a lista de usuários e o id do usuário para verificação. Sua função é exatamente verificar se o id recebido esta ou não na lista, indicando se o usuário já existe ou é novo.

In [ ]:
def verificarUsuario(id):
    listaUsersId = receberListaUser()
    if id in listaUsersId:
        return True
    else: return False

#### Verificar

Essa função não executa nenhuma tarefa, ela existe simplesmente para que qualquer mensagem recebida do telegram seja tratada e respondida da devida forma pelo resto do código. 

In [ ]:
def verificar(mensagem):
    return True


#### Funções de escuta

Amabas as funções abaixo possuem a mesma característica com uma única diferença, a primeria tem como objetivo receber qualquer mensagem do usuário, identificar se é um usuário novo ou existente, criar um objeto com as informações do usuário e chamar as funções que definem a mensagem recebida e ao final salva as alterações feitas no usuário. A segunda realiza a mesma função, porém ela é responsável por receber as mensagens vindas apenas dos botões exibidos durante a exceução.

In [ ]:
@bot.message_handler(func=verificar)
def receber(mensagem):

    id = mensagem.chat.id
    mensagemUser = mensagem.text

    if verificarUsuario(id):
        print('Usuario existente')
        usuario, linha = UserInfo(id)
        user = User(usuario[0], usuario[1], usuario[2], usuario[3], usuario[4], usuario[5], usuario[6], usuario[7])
    else:
        print('Usuario novo')
        user = User(id, 'Não existem frases', 'Não existem frases para traduzir, peça uma!', 'Básico', 0, '/OK', 'Frase', 'A')
        registrarUsuario(user)
        usuario, linha = UserInfo(id)


    
    Section(mensagemUser, user)
    AlterarUsuario(user, linha)

In [ ]:
@bot.callback_query_handler(func=lambda call:True)
def receber_btn(callback):
    id = callback.message.chat.id
    mensagemUser = callback.data

    if verificarUsuario(id):
        print('Usuario existente')
        usuario, linha = UserInfo(id)
        user = User(usuario[0], usuario[1], usuario[2], usuario[3], usuario[4], usuario[5], usuario[6], usuario[7])
    else:
        print('Usuario novo')
        user = User(id, 'Não existem frases', 'Não existem frases para traduzir, peça uma!', 'Básico', 0, '/OK', 'Frase', 'A')
        registrarUsuario(user)
        usuario, linha = UserInfo(id)

    Section(mensagemUser, user)
    AlterarUsuario(user, linha)

#### Bot Polling

Esse pequeno comando é responsável por deixar o bot em looping, ou seja, sempre 'escutando' as mensagem do telegram.

In [ ]:
bot.polling()